In [1]:
import pandas as pd

In [2]:
from tf.app import use
A = use('etcbc/bhsa', hoist=globals())

A.load(['g_prs', 'g_nme', 'g_pfm', 'g_vbs', 'g_vbe'])

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


True

In [3]:
valid_suff_chars = {'H', 'J', 'K', 'M', 'N', 'W'}

In [4]:
lexemes_with_suff = set()
for w in F.otype.s('word'):
    
    is_perfect = F.vt.v(w) == 'perf' and F.ps.v(w) == 'p3' and F.nu.v(w) == 'pl'
    is_imperfect = F.vt.v(w) == 'impf' and F.ps.v(w) in {'p2', 'p3'} and F.nu.v(w) == 'pl' and F.gn.v(w) == 'm'
    is_imperative = F.vt.v(w) == 'impv' and F.nu.v(w) == 'pl' and F.gn.v(w) == 'm'
    
    if is_perfect or is_imperfect or is_imperative:
        suff = ''.join([char for char in F.g_prs.v(w) if char in valid_suff_chars])
        if suff:
            lexemes_with_suff.add(F.lex.v(w))
            
len(lexemes_with_suff)

212

In [39]:
vbe_set = set()

info_dict = {}

for w in F.otype.s('word'):
    
    is_perfect = F.vt.v(w) == 'perf' and F.ps.v(w) == 'p3' and F.nu.v(w) == 'pl'
    is_imperfect = F.vt.v(w) == 'impf' and F.ps.v(w) in {'p2', 'p3'} and F.nu.v(w) == 'pl' and F.gn.v(w) == 'm'
    is_imperative = F.vt.v(w) == 'impv' and F.nu.v(w) == 'pl' and F.gn.v(w) == 'm'
    lang = F.language.v(w)
    if is_perfect or is_imperfect or is_imperative and lang == 'Hebrew':
        suff = ''.join([char for char in F.g_prs.v(w) if char in valid_suff_chars])
        #print(F.lex_utf8.v(w))
        vbe = F.g_vbe.v(w)

        vbe_set.add(vbe)
        
        lex = F.lex.v(w)
        g_cons = F.g_cons.v(w)
        has_suffix = int(len(suff) > 0)
        bo, ch, ve = T.sectionFromNode(w)
        if F.qere.v(w) != None:
            continue
        if vbe in {'[','[H','[J', '[W.>', '[WJ'}:
            print(w, bo, ch, ve, vbe, F.g_cons.v(w), suff, F.qere.v(w))
        if lex in lexemes_with_suff:
            if vbe not in {'[U', '[W', '[W.', '[W.-',}:
                #print(w, bo, ch, ve, vbe, F.g_cons.v(w), suff, F.qere.v(w))
                continue
            
            g_cons_no_suff = F.g_cons.v(w).rstrip(suff)
            suff_has_w = int('W' in suff)
            verb_ends_on_w = int(g_cons_no_suff[-1] == 'W')
            if not g_cons_no_suff.endswith('WN'):
                info_dict[w] = [w, bo, ch, ve, lex, F.lex_utf8.v(w), g_cons, g_cons_no_suff, suff, 
                    F.vt.v(w), F.vs.v(w), F.gn.v(w), F.nu.v(w), F.ps.v(w), verb_ends_on_w, has_suffix, suff_has_w] 

118926 Joshua 10 24 [W.> HLKW>  None
221017 Isaiah 28 12 [W.> >BW>  None
239977 Jeremiah 10 5 [W.> JNFW>  None


In [40]:
vbe_set

{'[',
 '[H',
 '[J',
 '[ON',
 '[OW',
 '[OWN',
 '[TEM',
 '[U',
 '[UN',
 '[UN.',
 '[W',
 '[W.',
 '[W.-',
 '[W.>',
 '[W.N',
 '[W.N.',
 '[WJ',
 '[WN'}

In [41]:
len(info_dict)

5054

In [42]:
dat = pd.DataFrame(info_dict).T

dat.columns = ['tf_id', 'book', 'chapter', 'verse', 'lex', 'lex_hebrew', 'g_cons', 'g_cons_no_suff', 'suff', 'vt', 'vs', 
               'gn', 'nu', 'ps', 'verb_ends_on_w', 'has_suffix', 'w_in_suffix']
dat.head(20)

,tf_id,book,chapter,verse,lex,lex_hebrew,g_cons,g_cons_no_suff,suff,vt,vs,gn,nu,ps,verb_ends_on_w,has_suffix,w_in_suffix
424,424,Genesis,1,22,ML>[,מלא,ML>W,ML>W,,impv,qal,m,pl,p2,1,0,0
564,564,Genesis,1,28,ML>[,מלא,ML>W,ML>W,,impv,qal,m,pl,p2,1,0,0
569,569,Genesis,1,28,KBC[,כבשׁ,KBCH,KBC,H,impv,qal,m,pl,p2,0,1,0
1192,1192,Genesis,3,1,>KL[,אכל,T>KLW,T>KLW,,impf,qal,m,pl,p2,1,0,0
1224,1224,Genesis,3,3,>KL[,אכל,T>KLW,T>KLW,,impf,qal,m,pl,p2,1,0,0
1764,1764,Genesis,4,6,NPL[,נפל,NPLW,NPLW,,perf,qal,unknown,pl,p3,1,0,0
2676,2676,Genesis,6,4,BW>[,בוא,JB>W,JB>W,,impf,qal,m,pl,p3,1,0,0
3004,3004,Genesis,6,20,BW>[,בוא,JB>W,JB>W,,impf,qal,m,pl,p3,1,0,0
3199,3199,Genesis,7,9,BW>[,בוא,B>W,B>W,,perf,qal,unknown,pl,p3,1,0,0
3262,3262,Genesis,7,11,PTX[,פתח,NPTXW,NPTXW,,perf,nif,unknown,pl,p3,1,0,0


In [43]:
dat.to_csv('plural_verbs.csv', sep='\t')

In [44]:
pd.crosstab(dat.verb_ends_on_w, dat.has_suffix)

has_suffix,0,1
verb_ends_on_w,,
0,0,237
1,4482,335


In [45]:
pd.crosstab(dat.verb_ends_on_w, dat.w_in_suffix)

w_in_suffix,0,1
verb_ends_on_w,,
0,68,169
1,4817,0
